# Hotel Recommender System
Train and deploy a Hotel Recommender System using the public Expedia data from Kaggle competition https://www.kaggle.com/c/expedia-hotel-recommendations/overview

**The goal is to help Expedia visitors find their dream hotel!**

Download data from https://www.kaggle.com/c/expedia-hotel-recommendations/data and unzip the downloaded file.

In [5]:
import json
import time

import boto3 

session = boto3.Session(profile_name='personalize')
iam = session.client("iam")

role_name = "PersonalizeRole"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::296654805457:role/PersonalizeRole


In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('./expedia-hotel-recommendations/train.csv')

In [8]:
df.head(100)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2014-01-08 14:09:47,2,3,66,462,41898,2454.8588,1482,0,0,...,0,2,28494,6,0,1,2,50,680,95
96,2014-01-08 14:15:40,2,3,66,462,41898,2455.2272,1482,0,0,...,0,1,28494,6,0,4,2,50,680,77
97,2014-01-08 14:18:31,2,3,66,462,41898,2455.2272,1482,0,0,...,0,1,28494,6,0,4,2,50,680,77
98,2014-01-08 14:30:25,2,3,66,462,41898,2455.2272,1482,0,0,...,0,2,28494,6,0,2,2,50,680,77


In [9]:
df.dtypes

date_time                     object
site_name                      int64
posa_continent                 int64
user_location_country          int64
user_location_region           int64
user_location_city             int64
orig_destination_distance    float64
user_id                        int64
is_mobile                      int64
is_package                     int64
channel                        int64
srch_ci                       object
srch_co                       object
srch_adults_cnt                int64
srch_children_cnt              int64
srch_rm_cnt                    int64
srch_destination_id            int64
srch_destination_type_id       int64
is_booking                     int64
cnt                            int64
hotel_continent                int64
hotel_country                  int64
hotel_market                   int64
hotel_cluster                  int64
dtype: object

In [10]:
df['date_time'].head(10)

0    2014-08-11 07:46:59
1    2014-08-11 08:22:12
2    2014-08-11 08:24:33
3    2014-08-09 18:05:16
4    2014-08-09 18:08:18
5    2014-08-09 18:13:12
6    2014-07-16 09:42:23
7    2014-07-16 09:45:48
8    2014-07-16 09:52:11
9    2014-07-16 09:55:24
Name: date_time, dtype: object

In [12]:
import numpy as np

df['ts'] = pd.to_datetime(df['date_time'], format="%Y-%m-%d %H:%M:%S").values.astype(np.int64) // 10**6

In [13]:
df.dtypes

date_time                     object
site_name                      int64
posa_continent                 int64
user_location_country          int64
user_location_region           int64
user_location_city             int64
orig_destination_distance    float64
user_id                        int64
is_mobile                      int64
is_package                     int64
channel                        int64
srch_ci                       object
srch_co                       object
srch_adults_cnt                int64
srch_children_cnt              int64
srch_rm_cnt                    int64
srch_destination_id            int64
srch_destination_type_id       int64
is_booking                     int64
cnt                            int64
hotel_continent                int64
hotel_country                  int64
hotel_market                   int64
hotel_cluster                  int64
ts                             int64
dtype: object

In [14]:
df.head(10)

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,ts
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,1,8250,1,0,3,2,50,628,1,1407743219000
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,1,8250,1,1,1,2,50,628,1,1407745332000
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,1,8250,1,0,1,2,50,628,1,1407745473000
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,1,14984,1,0,1,2,50,1457,80,1407607516000
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,1,14984,1,0,1,2,50,1457,21,1407607698000
5,2014-08-09 18:13:12,2,3,66,442,35390,911.5142,93,0,0,...,1,14984,1,0,1,2,50,1457,92,1407607992000
6,2014-07-16 09:42:23,2,3,66,189,10067,NaN,501,0,0,...,1,8267,1,0,2,2,50,675,41,1405503743000
7,2014-07-16 09:45:48,2,3,66,189,10067,NaN,501,0,1,...,1,8267,1,0,1,2,50,675,41,1405503948000
8,2014-07-16 09:52:11,2,3,66,189,10067,NaN,501,0,0,...,1,8267,1,0,1,2,50,675,69,1405504331000
9,2014-07-16 09:55:24,2,3,66,189,10067,NaN,501,0,0,...,1,8267,1,0,1,2,50,675,70,1405504524000


### Build the user-item interactions data set

In [15]:
df_subset = df[['user_id', 'hotel_cluster', 'ts']]

In [16]:
df_subset.head(10)

,user_id,hotel_cluster,ts
0,12,1,1407743219000
1,12,1,1407745332000
2,12,1,1407745473000
3,93,80,1407607516000
4,93,21,1407607698000
5,93,92,1407607992000
6,501,41,1405503743000
7,501,41,1405503948000
8,501,69,1405504331000
9,501,70,1405504524000


In [17]:
df_subset.columns = ['USER_ID','ITEM_ID', 'TIMESTAMP']

In [18]:
session = boto3.Session(profile_name='personalize')  # replace with an aws profile with access to S3 and Personalize
personalize = session.client('personalize', region_name='us-east-1')
personalize_runtime = session.client('personalize-runtime', region_name='us-east-1')

In [19]:
bucket = "personalize-hotels"  # replace with the name of your S3 bucket. Make sure the bucket is already created.
filename = "hotels-interactions.csv"  # replace with a name that you want to save the dataset under

In [20]:
# Save user-item interactions data set in a file locally
df_subset.to_csv(filename, index=False)

# Upload user-item interactions data set file to S3
session.resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Define user-item interactions data set schema in Amazon Personalize

In [21]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "twitch-hotel-recommender-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:296654805457:schema/twitch-hotel-recommender-schema",
  "ResponseMetadata": {
    "RequestId": "5cb6d209-e05c-477e-b0b8-7e0f3f7d8285",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 12:17:14 GMT",
      "x-amzn-requestid": "5cb6d209-e05c-477e-b0b8-7e0f3f7d8285",
      "content-length": "97",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Create data set group in Amazon Personalize

In [22]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "twitch-hotel-recommender-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:296654805457:dataset-group/twitch-hotel-recommender-group",
  "ResponseMetadata": {
    "RequestId": "c30b4e23-1fa9-47cd-8b6c-b9de1dbf302f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 12:17:15 GMT",
      "x-amzn-requestid": "c30b4e23-1fa9-47cd-8b6c-b9de1dbf302f",
      "content-length": "109",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [23]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


### Create data set in the group in Amazon Personalize

In [24]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "twitch-hotel-recommender-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:296654805457:dataset/twitch-hotel-recommender-group/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "76e5565a-432f-494f-878a-c54c780d1f37",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 12:18:15 GMT",
      "x-amzn-requestid": "76e5565a-432f-494f-878a-c54c780d1f37",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [26]:
role_arn = "arn:aws:iam::296654805457:role/Personalize"  # replace with a Role that has access to Personalize

create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "twitch-hotel-recommender-dataset-import-job",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:296654805457:dataset-import-job/twitch-hotel-recommender-dataset-import-job",
  "ResponseMetadata": {
    "RequestId": "91064ae0-3254-4be3-973a-8427ed8eb2e4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 12:22:24 GMT",
      "x-amzn-requestid": "91064ae0-3254-4be3-973a-8427ed8eb2e4",
      "content-length": "131",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [27]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE


### List recommender algorithms/recipes available in Amazon Personalize

In [28]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 1, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 7, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 1, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 7, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 1, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 7, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

In [29]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

### Create solution in Amazon Personalize
In other words, let's train the hotel-recommender system!

In [30]:
create_solution_response = personalize.create_solution(
    name = "twitch-hotel-recommender-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:296654805457:solution/twitch-hotel-recommender-solution",
  "ResponseMetadata": {
    "RequestId": "a414edeb-ddfc-4c31-8016-5a3fd74ddc47",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 12:38:34 GMT",
      "x-amzn-requestid": "a414edeb-ddfc-4c31-8016-5a3fd74ddc47",
      "content-length": "103",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [31]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:296654805457:solution/twitch-hotel-recommender-solution/c1af2b58",
  "ResponseMetadata": {
    "RequestId": "b53b318d-ac02-4388-a4ff-2acdae4286b5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 12:38:34 GMT",
      "x-amzn-requestid": "b53b318d-ac02-4388-a4ff-2acdae4286b5",
      "content-length": "119",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [32]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

### Time to retrieve accuracy metrics of the trained recommender system model!

In [33]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:296654805457:solution/twitch-hotel-recommender-solution/c1af2b58",
  "metrics": {
    "coverage": 0.9901,
    "mean_reciprocal_rank_at_25": 0.5177,
    "normalized_discounted_cumulative_gain_at_10": 0.5381,
    "normalized_discounted_cumulative_gain_at_25": 0.5924,
    "normalized_discounted_cumulative_gain_at_5": 0.4959,
    "precision_at_10": 0.1067,
    "precision_at_25": 0.0602,
    "precision_at_5": 0.1626
  },
  "ResponseMetadata": {
    "RequestId": "c2490d34-a508-45ea-837d-7eb30172dc94",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 14:08:02 GMT",
      "x-amzn-requestid": "c2490d34-a508-45ea-837d-7eb30172dc94",
      "content-length": "417",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Let's deploy the trained recommender model!

In [34]:
create_campaign_response = personalize.create_campaign(
    name = "twitch-hotel-recommender-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:296654805457:campaign/twitch-hotel-recommender-campaign",
  "ResponseMetadata": {
    "RequestId": "120e44a9-9b3c-4d50-ba85-9e1bf21ab21e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 25 Jun 2020 14:08:03 GMT",
      "x-amzn-requestid": "120e44a9-9b3c-4d50-ba85-9e1bf21ab21e",
      "content-length": "103",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [35]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE PENDING
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


In [36]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = '93'
)

In [37]:
get_recommendations_response

{'ResponseMetadata': {'RequestId': 'fded2d93-e76e-4908-ad49-f5553c42ed46',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 25 Jun 2020 14:17:05 GMT',
   'x-amzn-requestid': 'fded2d93-e76e-4908-ad49-f5553c42ed46',
   'content-length': '1360',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'itemList': [{'itemId': '92'},
  {'itemId': '80'},
  {'itemId': '21'},
  {'itemId': '96'},
  {'itemId': '66'},
  {'itemId': '26'},
  {'itemId': '64'},
  {'itemId': '44'},
  {'itemId': '84'},
  {'itemId': '0'},
  {'itemId': '65'},
  {'itemId': '86'},
  {'itemId': '58'},
  {'itemId': '63'},
  {'itemId': '97'},
  {'itemId': '52'},
  {'itemId': '25'},
  {'itemId': '69'},
  {'itemId': '98'},
  {'itemId': '22'},
  {'itemId': '73'},
  {'itemId': '19'},
  {'itemId': '41'},
  {'itemId': '67'},
  {'itemId': '34'}]}